In [1]:
#ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from numba import cuda,jit
import cupy as cp
import bs4
import chromadb
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult
import sys
import time
import json
import requests

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
model_local = ChatOllama(model='llama3.2')
#client1 = chromadb.PersistentClient(path="./vectorstore1")
API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-mpnet-base-v2"
headers = {"Authorization": "Bearer API_TOKEN"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
    "inputs": ["this is a sentence", "this is another sentence"]
})
# Output is a list of 2 embeddings, each of 768 values.

In [12]:
# Training
def load_document(path): 
    document_loader = PyPDFDirectoryLoader(path)
    return document_loader.load()
client = chromadb.Client()

In [6]:
#collection = client.create_collection(name="rag-chroma1")

In [7]:
# t=Training
path= 'data'
document = load_document(path)
document[1]

Document(metadata={'source': 'data\\7434402.pdf', 'page': 1}, page_content='IUIUIUIUIUIUIUIUIUIUIUIUIUIUIU!UIUIUIUIUIUIOlO!CJT\nTheGanges\ninMythandHistory')

In [8]:
# Training
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=10000, chunk_overlap=100)
doc_split = text_splitter.split_documents(document)

In [9]:

with cp.cuda.Device(0):
    vectorstore = Chroma.from_documents(
        documents=doc_split,
        embedding=OllamaEmbeddings(model='nomic-embed-text'),
        persist_directory="./chroma_db"
    )

KeyboardInterrupt: 

In [ ]:

vectorstore.persist()

c:\Users\KIRAN\Documents\ML\Dhara\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [3]:
#streaming_callback = StreamingStdOutCallbackHandler()
embeddings = OllamaEmbeddings(model='nomic-embed-text')
loaded_vectorstore = Chroma(persist_directory="./chroma_db1", embedding_function=embeddings)
loaded_vectorstore.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [4]:
#retriever = vectorstore.as_retriever()
retriever = loaded_vectorstore.as_retriever()

In [5]:
'''before_rag_template = "{topic}"
print("Before using RAG:")
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic":"What is proposed in the paper Attention is All You Need ?"}))'''

'before_rag_template = "{topic}"\nprint("Before using RAG:")\nbefore_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)\nbefore_rag_chain = before_rag_prompt | model_local | StrOutputParser()\nprint(before_rag_chain.invoke({"topic":"What is proposed in the paper Attention is All You Need ?"}))'

In [6]:
after_rag_template = """answer the question based on the following context:
{context}
Question:{question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
     | after_rag_prompt
     | model_local
     | StrOutputParser()
)

In [7]:
a  = after_rag_chain.invoke("reasons behind the pollution of ganga")


In [8]:
def evaluate_rag_result(generated_response, query, expected_facts):
    """
    Evaluate the RAG result based on various criteria.
    
    Parameters:
    - generated_response (str): The output from the RAG model.
    - query (str): The original input query.
    - expected_facts (list): A list of facts that should be present in the response.
    
    Returns:
    - evaluation (dict): A dictionary containing scores for each criterion.
    """
    
    evaluation = {
        "Relevance": evaluate_relevance(generated_response, query),
        "Accuracy": evaluate_accuracy(generated_response, expected_facts),
        "Coherence": evaluate_coherence(generated_response),
        "Completeness": evaluate_completeness(generated_response, expected_facts),
        "Conciseness": evaluate_conciseness(generated_response),
        "Engagement": evaluate_engagement(generated_response),
        "Contextual Understanding": evaluate_contextual_understanding(generated_response)
    }
    
    return evaluation

def evaluate_relevance(response, query):
    return 1 if query.lower() in response.lower() else 0

def evaluate_accuracy(response, expected_facts):
    return sum(1 for fact in expected_facts if fact.lower() in response.lower()) / len(expected_facts)

def evaluate_coherence(response):
    sentences = response.split('.')
    return len(sentences) / max(1, len(sentences))

def evaluate_completeness(response, expected_facts):
    return sum(1 for fact in expected_facts if fact.lower() in response.lower()) / len(expected_facts)

def evaluate_conciseness(response):
    word_count = len(response.split())
    return 1 if word_count < 100 else 0

def evaluate_engagement(response):
    return 1 if '?' in response or 'interesting' in response else 0

def evaluate_contextual_understanding(response):
    return 1 if 'context' in response else 0




In [9]:
a

'The Ganga River, one of the most sacred and revered rivers in Hinduism, has been facing severe pollution issues for decades. The main reasons behind the pollution of the Ganga are:\n\n1. **Industrial effluents**: The river receives industrial waste from tanneries, dye works, and other industries located along its banks. These industries release toxic chemicals, heavy metals, and pesticides into the water.\n2. **Domestic sewage**: The rapid urbanization of Haridwar and Varanasi has led to a significant increase in domestic sewage, which is released directly into the river. This sewage contains human waste, industrial effluents, and other pollutants.\n3. **Agricultural runoff**: Agricultural activities in the surrounding areas lead to nutrient-rich runoff, which fertilizes the Ganga River and contributes to its pollution.\n4. **Unregulated cremation grounds**: In some parts of Haridwar and Varanasi, cremation grounds are located directly on the riverbanks, leading to a significant relea

In [10]:
def evallama(query,expected_facts_input):
    # ANSI escape codes for coloring
    GREEN = "\033[92m"
    RED = "\033[91m"
    RESET = "\033[0m"

    # Interactive part of the script
    print("RAG Result Evaluation\n" + "-" * 30)
    generated_response = after_rag_chain.invoke(query)
    expected_facts = [fact.strip() for fact in expected_facts_input.split(',')]
    evaluation_results = evaluate_rag_result(generated_response, query, expected_facts)
    print(generated_response)
    print("RAG Result Evaluation\n" + "-" * 30)
    # Print evaluation results with colors
    print("\nEvaluation Results:")
    for criterion, score in evaluation_results.items():
        if score >= 0.5:
            print(f"{GREEN}{criterion}: {score:.2f}{RESET}")
        else:
            print(f"{RED}{criterion}: {score:.2f}{RESET}")

In [11]:
evallama("what is ganga","ganga is a famous river in india")

RAG Result Evaluation
------------------------------
Based on general knowledge, Ganga (also known as the Ganges River) is one of the most sacred rivers in Hinduism and a vital source of water, nutrients, and spiritual energy for millions of people in India. Is that what you were referring to?
RAG Result Evaluation
------------------------------

Evaluation Results:
Relevance: 0.00
Accuracy: 0.00
Coherence: 1.00
Completeness: 0.00
Conciseness: 1.00
Engagement: 1.00
Contextual Understanding: 0.00


In [12]:
def evaluate_rag_result(generated_response, query, expected_facts):
    """
    Evaluate the RAG result based on various criteria.
    
    Parameters:
    - generated_response (str): The output from the RAG model.
    - query (str): The original input query.
    - expected_facts (list): A list of facts that should be present in the response.
    
    Returns:
    - evaluation (dict): A dictionary containing scores for each criterion.
    """

    evaluation = {
        "Relevance": evaluate_relevance(generated_response, query),
        "Accuracy": evaluate_accuracy(generated_response, expected_facts),
        "Coherence": evaluate_coherence(generated_response),
        "Completeness": evaluate_completeness(generated_response, expected_facts),
        "Conciseness": evaluate_conciseness(generated_response),
        "Engagement": evaluate_engagement(generated_response),
        "Contextual Understanding": evaluate_contextual_understanding(generated_response)
    }

    return evaluation

def evaluate_relevance(response, query):
    return 1 if query.lower() in response.lower() else 0

def evaluate_accuracy(response, expected_facts):
    return sum(1 for fact in expected_facts if fact.lower() in response.lower()) / len(expected_facts)

def evaluate_coherence(response):
    sentences = response.split('.')
    return len(sentences) / max(1, len(sentences))

def evaluate_completeness(response, expected_facts):
    return sum(1 for fact in expected_facts if fact.lower() in response.lower()) / len(expected_facts)

def evaluate_conciseness(response):
    word_count = len(response.split())
    return 1 if word_count < 100 else 0

def evaluate_engagement(response):
    return 1 if '?' in response or 'interesting' in response else 0

def evaluate_contextual_understanding(response):
    return 1 if 'context' in response else 0

def evallama(query,expected_facts_input):
    # ANSI escape codes for coloring
    GREEN = "\033[92m"
    RED = "\033[91m"
    RESET = "\033[0m"

    # Interactive part of the script
    print("RAG Result Evaluation\n" + "-" * 30)
    generated_response = after_rag_chain.invoke(query)
    expected_facts = [fact.strip() for fact in expected_facts_input.split(',')]
    evaluation_results = evaluate_rag_result(generated_response, query, expected_facts)
    print(generated_response)
    print("RAG Result Evaluation\n" + "-" * 30)
    # Print evaluation results with colors
    print("\nEvaluation Results:")
    for criterion, score in evaluation_results.items():
        if score >= 0.5:
            print(f"{GREEN}{criterion}: {score:.2f}{RESET}")
        else:
            print(f"{RED}{criterion}: {score:.2f}{RESET}")

In [15]:
evallama("what is ganga","""Based on general knowledge, I can provide several possible answers:1. **River Ganga**: In Hindu mythology and culture, Ganga refers to the Ganges River, a sacred river in India that is considered one of the most holy rivers in Hinduism.2. **Ganga (city)**: Ganga is also a city located in South Africa, which was previously known as Port Elizabeth. It's now part of the Nelson Mandela Bay Metropolitan Municipality.3. **Bhagwan Shree Krishna's Ganga**: In Hindu mythology, Ganga is also another name for Bhagwan Shree Krishna's consort, Radha's sister, or a river that served as a mode of transportation and an essential resource in ancient times.However, without more context, it's difficult to determine which "Ganga" you are referring to. If you could provide more information or clarify your question, I'd be happy to try and give a more specific answer!RAG Result Evaluation""")


RAG Result Evaluation
------------------------------
Based on the provided context, I'm assuming you're referring to the Ganga river. The Ganga (also known as the Ganges) is a major river in India and Bangladesh, considered one of the most sacred rivers in Hinduism. It originates in the Himalayas and flows through several Indian states before emptying into the Bay of Bengal.
RAG Result Evaluation
------------------------------

Evaluation Results:
Relevance: 0.00
Accuracy: 0.00
Coherence: 1.00
Completeness: 0.00
Conciseness: 1.00
Engagement: 0.00
Contextual Understanding: 1.00
